In [ ]:
import torch.optim as optim
import h5py
import numpy as np

from torch.utils.data import random_split
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")
def train_and_validate(model, train_loader, val_loader, epochs, device):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.00001)

    model.to(device)

    for epoch in range(epochs):
        # Training Phase
        model.train()
        running_loss = 0.0
        for i, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 10 == 0:
                print(f'Epoch {epoch + 1}, Batch {i}, Train Loss: {running_loss / 10:.4f}')
                running_loss = 0.0

        # Validation Phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        print(f'Epoch {epoch + 1}, Validation Loss: {val_loss:.4f}')

# Setup
file_path = '/notebooks/mass_spectra_pairs.hdf5'
batch_size = 16
train_loader, val_loader = get_data_loaders(file_path, batch_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Runet()
train_and_validate(model, train_loader, val_loader, epochs=100, device=device)
save_model(model, 'model_v2.pth')
